<h1><center>Capstone Project</center></h1>

This is the repository for the IBM data science professional certificate needed for the capstone project 
Here I will try to cluster similar neghborhoods in the city of Toronto.


In [2]:
#importing the necessary libraries

import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
#getting the html with urlopen
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url)

In [4]:
# create a BeautifulSoup object from the html file
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

Now it is time to extract the table from the BeautifulSoup object!

In [5]:
#find all the row elements
rows = soup.find_all('tr')
print(rows[:10])  #print to check things are going like planned

[<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>, <tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>, <tr>
<td>M6A</td>
<td><a href="/wiki/North_York" ti

In [6]:
#get a list composed by all the rows in the rows element
list_rows = []
for row in rows:
    row_td = row.find_all('td')
    str_cells = str(row_td)
    cleantext = BeautifulSoup(str_cells, "lxml").get_text()
    list_rows.append(cleantext)
print(list_rows[0:10])
print (list_rows[-10:])
    

['[]', '[M1A, Not assigned, Not assigned\n]', '[M2A, Not assigned, Not assigned\n]', '[M3A, North York, Parkwoods\n]', '[M4A, North York, Victoria Village\n]', '[M5A, Downtown Toronto, Harbourfront\n]', '[M5A, Downtown Toronto, Regent Park\n]', '[M6A, North York, Lawrence Heights\n]', '[M6A, North York, Lawrence Manor\n]', "[M7A, Queen's Park, Not assigned\n]"]
['[M8Z, Etobicoke, Mimico NW\n]', '[M8Z, Etobicoke, The Queensway West\n]', '[M8Z, Etobicoke, Royal York South West\n]', '[M8Z, Etobicoke, South of Bloor\n]', '[M9Z, Not assigned, Not assigned\n]', '[]', '[\n, \n]', '[\n\n\nNL\n\nNS\n\nPE\n\nNB\n\nQC\n\nON\n\nMB\n\nSK\n\nAB\n\nBC\n\nNU/NT\n\nYT\n\n\nA\n\nB\n\nC\n\nE\n\nG\n\nH\n\nJ\n\nK\n\nL\n\nM\n\nN\n\nP\n\nR\n\nS\n\nT\n\nV\n\nX\n\nY\n\n, NL\n, NS\n, PE\n, NB\n, QC\n, ON\n, MB\n, SK\n, AB\n, BC\n, NU/NT\n, YT\n, A\n, B\n, C\n, E\n, G\n, H\n, J\n, K\n, L\n, M\n, N\n, P\n, R\n, S\n, T\n, V\n, X\n, Y\n]', '[NL\n, NS\n, PE\n, NB\n, QC\n, ON\n, MB\n, SK\n, AB\n, BC\n, NU/NT\n, YT\n]

In [7]:
#get rid of undesired elements inside the list
list_rows = list_rows[1:-5]

In [8]:
#creating a dataframe from the list
df = pd.DataFrame(list_rows)
df.head()

,0
0,"[M1A, Not assigned, Not assigned\n]"
1,"[M2A, Not assigned, Not assigned\n]"
2,"[M3A, North York, Parkwoods\n]"
3,"[M4A, North York, Victoria Village\n]"
4,"[M5A, Downtown Toronto, Harbourfront\n]"


## Data Manipulation and Cleaning
Now that we have a pandas DataFrame, it is time to perform operations in order to obtain a ready-to-use DataFrame so we can cluster the Toronto neighborhoods

1. Lets start by splitting the data into the columns

In [9]:
#splitting column 0 using the separator ','
df1 = df[0].str.split(',', expand=True)
df1.head(10)

,0,1,2
0,[M1A,Not assigned,Not assigned\n]
1,[M2A,Not assigned,Not assigned\n]
2,[M3A,North York,Parkwoods\n]
3,[M4A,North York,Victoria Village\n]
4,[M5A,Downtown Toronto,Harbourfront\n]
5,[M5A,Downtown Toronto,Regent Park\n]
6,[M6A,North York,Lawrence Heights\n]
7,[M6A,North York,Lawrence Manor\n]
8,[M7A,Queen's Park,Not assigned\n]
9,[M8A,Not assigned,Not assigned\n]


And lets get rid of the unwanted characters such as '['

In [10]:
#using the strip method to remove unwanted characters
df1[0] = df1[0].str.strip('[')
df1[2] = df1[2].str.strip('\n]')
df1.head(10)

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


It looks like our table is missing the column headers, lets change that!

Finally, lets rename our columns in df1

In [11]:
df1.columns = ['PostalCode','Borough','Neighborhood']
df1.head()
 

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


2. Now it is time to format the table so it is in line with the required output.

Lets start by removing the rows that do not contain any information in the 'Borough' column

In [12]:
df1 = df1[~df1['Borough'].str.contains('Not assigned')]  
df1.shape


(211, 3)

In [13]:
dfprueba= df1  #This is just to aviod screwing up the original DataFrame in case things do not go as expected


In [14]:

dfprueba.at[8,'Neighborhood'] = "Queen's Park"   #Assign the same name as the borough

In [15]:
new_dict = {'PostalCode': [], 'Borough':[], 'Neighborhood':[]}
for postalcode in dfprueba['PostalCode'].unique():
    df_temp = dfprueba[dfprueba['PostalCode']==postalcode]
    neighbors= df_temp['Neighborhood'].str.cat(sep = ',')
    borough = df_temp.Borough.values[0]
    new_dict['PostalCode'].append(postalcode)
    new_dict['Borough'].append(borough)
    new_dict['Neighborhood'].append(neighbors)
toronto_df = pd.DataFrame(new_dict)
toronto_df.shape
    

(103, 3)

In [16]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
coordinates.rename( columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
result = pd.merge(toronto_df, coordinates, on='PostalCode')
result.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
